### Web Scraper to generate hisotrical weather data

In [107]:
import requests
import pprint
from pymongo import MongoClient
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import selenium
import numpy as np

First, clean up my data in order to identify which dates to scrape weather on <br/> Pulling historic weather data from Pheonix, AZ

In [3]:
'''
key items:

date
<div _ngcontent-app-root-c6="" class="date ng-star-inserted">1</div>

Weather description for the day
class = <div _ngcontent-app-root-c6="" class="phrase ng-star-inserted">mostly sunny</div>

Hi
<span _ngcontent-app-root-c6="" class="hi">80°</span>

Low
<span _ngcontent-app-root-c6="" class="low">61°</span>

Rain (in inches)
<span _ngcontent-app-root-c12="" class="wu-value wu-value-to">0.21</span>

FOCUS HERE
Phoenix, AZ
https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-4 
- date will be the changing variable
'''


'\nkey items:\n\ndate\n<div _ngcontent-app-root-c6="" class="date ng-star-inserted">1</div>\n\nWeather description for the day\nclass = <div _ngcontent-app-root-c6="" class="phrase ng-star-inserted">mostly sunny</div>\n\nHi\n<span _ngcontent-app-root-c6="" class="hi">80°</span>\n\nLow\n<span _ngcontent-app-root-c6="" class="low">61°</span>\n\nRain (in inches)\n<span _ngcontent-app-root-c12="" class="wu-value wu-value-to">0.21</span>\n\nFOCUS HERE\nPhoenix, AZ\nhttps://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-4 \n- date will be the changing variable\n'

### <center> Create a List of URL's to pull data from <center>

In [14]:
# Seems like I'll need to set up a web scraper for each year and have the month values change

url_lst_2019 = []

def web_scraper_links():
    url = 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1'
    for x in range(1,13):
        url_lst_2019.append(f"https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-{x}")
        
        
web_scraper_links()
# Above function worked just fine. Now have all 2019 weather data links

In [15]:
url_lst_2019

['https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-2',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-3',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-4',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-5',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-6',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-7',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-8',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-9',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-10',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-11',
 'https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-12']

In [74]:
# This starts up my automated browser
driver = webdriver.Safari()

In [75]:
# driver.get("https://www.wunderground.com/history/monthly/us/az/phoenix/KPHX/date/2019-1")
driver.get('https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1')

In [76]:
#data = driver.find_element_by_class_name("days.ng-star-inserted")
# date_data = driver.find_element_by_class_name("date.ng-star-inserted")
# description_data = driver.find_element_by_class_name('phrase.ng-star-inserted')
# hi_data = driver.find_element_by_class_name('hi')
# lo_data = driver.find_element_by_class_name('low')
# rain_data = driver.find_element_by_class_name('test-false.wu-unit.wu-unit-rain.ng-star-inserted')

In [77]:
daily_data = driver.find_elements_by_class_name('calendar-day.current-month.history.ng-star-inserted')

In [78]:
for x in daily_data:
    print(
        x.find_element_by_class_name("date.ng-star-inserted").text, 
        x.find_element_by_class_name("phrase.ng-star-inserted").text,
        x.find_element_by_class_name("hi").text,
        x.find_element_by_class_name("low").text,
        x.find_element_by_class_name("wu-value.wu-value-to").text
         )

1 mostly cloudy 50° 37° 0.14
2 mostly sunny 51° 30° 0
3 mostly sunny 56° 33° 0
4 mostly sunny 66° 37° 0
5 cloudy 67° 46° 0
6 cloudy 55° 46° 0.48
7 partly cloudy 62° 43° 0.07
8 mostly sunny 71° 43° 0
9 cloudy 72° 49° 0
10 cloudy 67° 50° 0
11 mostly sunny 65° 46° 0
12 mostly cloudy 68° 49° 0
13 mostly cloudy 62° 48° 0.04
14 cloudy 68° 51° 0
15 cloudy 65° 52° 0.03
16 partly cloudy 67° 51° 0.12
17 cloudy 69° 48° 0
18 mostly cloudy 69° 53° 0
19 partly cloudy 71° 45° 0
20 cloudy 74° 46° 0
21 partly cloudy 67° 49° 0
22 mostly sunny 63° 44° 0
23 mostly sunny 61° 41° 0
24 mostly sunny 68° 41° 0
25 mostly sunny 68° 40° 0
26 mostly sunny 71° 42° 0
27 mostly cloudy 73° 45° 0
28 cloudy 72° 47° 0
29 cloudy 71° 52° 0
30 cloudy 73° 56° 0
31 mostly cloudy 72° 51° 0


In [90]:
def get_weather_data():
    driver = webdriver.Safari()
    # change the hyperlink to a variable so I can iterate through my list of lists
    driver.get('https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2019-1')
    time.sleep(4)
    daily_data = driver.find_elements_by_class_name('calendar-day.current-month.history.ng-star-inserted')
    dates = []
    conditions = []
    his = []
    lows = []
    rain_ins = []
    for x in daily_data:
        date = x.find_element_by_class_name("date.ng-star-inserted").text
        dates.append(date)
        condition = x.find_element_by_class_name("phrase.ng-star-inserted").text
        conditions.append(condition)
        hi = x.find_element_by_class_name("hi").text
        his.append(hi)
        low = x.find_element_by_class_name("low").text
        lows.append(low)
        rain_in = x.find_element_by_class_name("wu-value.wu-value-to").text
        rain_ins.append(rain_in)
    df = pd.DataFrame()
    return dates, conditions, his, lows, rain_ins

In [108]:
dates, conditions, his, lows, rain_ins = get_weather_data()

In [109]:
lst = [dates, conditions, his, lows, rain_ins]

In [111]:
jan_weather_data = pd.DataFrame(lst) #, columns=['dates', 'conditions', 'his', 'lows', 'rain_ins'])

In [114]:
jan_weather_data = jan_weather_data.T

In [116]:
jan_weather_data.columns = ['Date', 'Condition', 'Hi', 'Low', 'Rain(inches)']

In [133]:
jan_weather_data['Indicator'] = 0

In [158]:
jan_weather_data['Indicator'] = jan_weather_data['Condition'].apply(lambda x: 0 if x == ("cloudy" or "mostly cloudy") else 1)

In [159]:
jan_weather_data

,Date,Condition,Hi,Low,Rain(inches),Indicator
0,1,mostly cloudy,50°,37°,0.14,1
1,2,mostly sunny,51°,30°,0,1
2,3,mostly sunny,56°,33°,0,1
3,4,mostly sunny,66°,37°,0,1
4,5,cloudy,67°,46°,0,0
5,6,cloudy,55°,46°,0.48,0
6,7,partly cloudy,62°,43°,0.07,1
7,8,mostly sunny,71°,43°,0,1
8,9,cloudy,72°,49°,0,0
9,10,cloudy,67°,50°,0,0
